In [1]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 31.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993243 sha256=f210675d9d5b634a1a5383019652c5a4a59c1bc29319cdb5f94449e5534c6232
  Stored in directory: /root/.cache/pip/wheels/d1/c1/d9/7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect


In [2]:
import pandas as pd
from langdetect import detect
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [3]:
def detect_language(text):
    try:
        lang = detect(text)
    except:
        lang = 'unknown'
    return lang

In [4]:
df = pd.read_excel('/content/ISKCON Data.xlsx')

In [5]:
df['new_text'] = df['REVIEW SUBJECT'].fillna(" ") + " " + df["text"].fillna(" ")
df['new_text'].str.strip()
df['length'] = df['new_text'].str.split().str.len()
df = df[df['length'] > 3]

In [6]:
pattern = r'[^a-zA-Z\s]'

# define a function to remove non-alphabetic characters from a text string
def remove_non_alpha(text):
    return re.sub(pattern, '', text)

# apply the function to the column you want to clean
df['text_lang'] = df['new_text'].apply(remove_non_alpha)

<ipython-input-6-b73e790a1738>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_lang'] = df['new_text'].apply(remove_non_alpha)


In [7]:
def remove_short_words(text):
    words = text.split()
    words = [word for word in words if len(word) >= 2]
    return ' '.join(words)

# apply the function to the column you want to clean
df['text_lang'] = df['text_lang'].apply(lambda x: remove_short_words(x))

<ipython-input-7-4bb6120c96b9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_lang'] = df['text_lang'].apply(lambda x: remove_short_words(x))


In [8]:
df['language'] = df['text_lang'].apply(detect_language)

In [9]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [10]:
sia = SentimentIntensityAnalyzer()
def sentiment_scores(text):
    pos_sent = []
    neg_sent = []
    sentences = nltk.sent_tokenize(text)
    pos_count = 0
    neg_count = 0
    for sentence in sentences:
        scores = sia.polarity_scores(sentence)
        if scores['compound'] > 0:
            pos_count += 1
            pos_sent.append(sentence)
        elif scores['compound'] < 0:
            neg_count += 1
            neg_sent.append(sentence)
    total_count = pos_count + neg_count
    if total_count > 0:
        pos_pct = pos_count / total_count * 100
        neg_pct = neg_count / total_count * 100
    else:
        pos_pct = 0
        neg_pct = 0
    return [pos_sent,pos_pct,neg_sent,neg_pct]

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [12]:
df['Sentiment'] = df['new_text'].apply(sentiment_scores)

In [13]:
df[['pos_sent','pos','neg_sent','neg']] = df['Sentiment'].apply(lambda x: pd.Series(x))

# drop the original list_column column
df.drop('Sentiment', axis=1, inplace=True)

In [14]:
!pip install NRCLex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 KB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for NRCLex: filename=NRCLex-3.0.0-py3-none-any.whl size=43327 sha256=9df40d35f45371887ebb63235a4338f43cc2a559922b5cc2af8fd07c0fa75313
  Stored in directory: /root/.cache/pip/wheels/68/c4/f2/c390dd3eac398fdf45f7a01c6516bc53fa7a9ab59c7d2ff518
Successfully built NRCLex


In [15]:
from nrclex import NRCLex

In [16]:
df['emotions'] = df.new_text.apply(lambda x: NRCLex(x).raw_emotion_scores)

In [19]:
df.to_csv('final.csv')

In [29]:
df['emotions']

0                               {'positive': 6, 'joy': 1}
1       {'positive': 4, 'trust': 3, 'anticipation': 4,...
2       {'positive': 10, 'trust': 8, 'joy': 6, 'anger'...
3                          {'negative': 1, 'positive': 1}
4       {'anticipation': 3, 'joy': 6, 'positive': 6, '...
                              ...                        
4642                {'joy': 1, 'positive': 1, 'trust': 1}
4643    {'anticipation': 1, 'joy': 2, 'positive': 3, '...
4646    {'joy': 2, 'positive': 4, 'surprise': 1, 'trus...
4649    {'positive': 2, 'anger': 1, 'negative': 1, 'an...
4650                            {'joy': 1, 'positive': 1}
Name: emotions, Length: 4274, dtype: object

In [49]:
positive = 0
for i in df['emotions']:
  try:
    positive = positive + i['positive'] 
  except:
    continue

In [50]:
negative = 0
for i in df['emotions']:
  try:
    negative = negative + i['negative'] 
  except:
    continue

In [51]:
joy = 0
for i in df['emotions']:
  try:
    joy = joy + i['joy'] 
  except:
    continue

In [52]:
trust = 0
for i in df['emotions']:
  try:
    trust = trust + i['trust'] 
  except:
    continue

In [53]:
anticipation = 0
for i in df['emotions']:
  try:
    anticipation = anticipation + i['anticipation'] 
  except:
    continue

In [54]:
surprise = 0
for i in df['emotions']:
  try:
    surprise = surprise + i['surprise'] 
  except:
    continue

In [55]:
anger = 0
for i in df['emotions']:
  try:
    anger = anger + i['anger'] 
  except:
    continue

In [56]:
sadness = 0
for i in df['emotions']:
  try:
    sadness = sadness + i['sadness'] 
  except:
    continue

In [57]:
disgust = 0
for i in df['emotions']:
  try:
    disgust = disgust + i['disgust'] 
  except:
    continue

In [58]:
fear = 0
for i in df['emotions']:
  try:
    fear = fear + i['fear'] 
  except:
    continue

In [69]:
pd.DataFrame([fear, anger, anticipation, trust, surprise, positive, negative, sadness, disgust, joy],['fear', 'anger', 'anticipation', 'trust', 'surprise', 'positive', 'negative', 'sadness', 'disgust', 'joy']).to_csv('emotions.csv')